In [67]:
# Column Information
# date : date of observation - the price is taken at 00:00:00 hours
# price : Price at the given date and time
# total_volume : volume of transactions on the given day
# market_cap : Market capitalization in USD

In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [69]:
# Load the CSV files into DataFrames
df1 = pd.read_csv('ethereum.csv')
df2 = pd.read_csv('solana.csv')

# Ensure the Date columns are in datetime format
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])

df1.rename(columns={'price': 'ethereum_price'}, inplace=True)
# Perform an inner join on the 'Date' column
merged_df = pd.merge(df2, df1[['date', 'ethereum_price']], on='date', how='inner')


# Save the merged DataFrame to a new CSV file if needed
merged_df.to_csv('merged_coins.csv', index=False)

# Display the merged DataFrame
print(merged_df)

           date       price  total_volume    market_cap coin_name  \
0    2020-04-11    0.957606  9.267267e+07  7.827652e+06    solana   
1    2020-04-12    0.784711  4.945869e+07  6.173608e+06    solana   
2    2020-04-13    0.875994  3.782101e+07  7.269680e+06    solana   
3    2020-04-14    0.786712  1.957921e+07  6.302364e+06    solana   
4    2020-04-15    0.666673  1.752353e+07  5.306221e+06    solana   
...         ...         ...           ...           ...       ...   
1442 2024-03-23  173.455636  4.530694e+09  7.673847e+10    solana   
1443 2024-03-24  173.453773  2.950137e+09  7.705263e+10    solana   
1444 2024-03-25  184.021864  2.903673e+09  8.189313e+10    solana   
1445 2024-03-26  188.996855  4.384505e+09  8.383933e+10    solana   
1446 2024-03-27  191.236145  4.010910e+09  8.483871e+10    solana   

      ethereum_price  
0         157.740158  
1         158.327878  
2         158.863826  
3         156.701359  
4         158.267151  
...              ...  
1442     3

In [70]:
# Extracting date features
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek
merged_df['month'] = merged_df['date'].dt.month
merged_df['day_of_month'] = merged_df['date'].dt.day
merged_df['year'] = merged_df['date'].dt.year


df = merged_df.drop(['date','coin_name'], axis=1)
df

df = df.drop_duplicates(keep='first')

for lag in range(1, 91):
    df[f'price_lag_{lag}'] = df['price'].shift(lag)
    
for lag in range(1, 7):
    df[f'price_month_lag_{lag}'] = df['price'].shift(lag * 7)

for lag in range(1, 60):
    df[f'ethereum_price_lag_{lag}'] = df['ethereum_price'].shift(lag)
    
for lag in range(1, 3):
    df[f'ethereum_price_month_lag_{lag}'] = df['ethereum_price'].shift(lag * 7)

print(df.shape)
df = df.dropna(axis=0)
print(df.dtypes)

(1447, 165)
price                         float64
total_volume                  float64
market_cap                    float64
ethereum_price                float64
day_of_week                     int32
                               ...   
ethereum_price_lag_57         float64
ethereum_price_lag_58         float64
ethereum_price_lag_59         float64
ethereum_price_month_lag_1    float64
ethereum_price_month_lag_2    float64
Length: 165, dtype: object


C:\Users\KYLEONEIL\AppData\Local\Temp\ipykernel_4216\2634338635.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'ethereum_price_lag_{lag}'] = df['ethereum_price'].shift(lag)
C:\Users\KYLEONEIL\AppData\Local\Temp\ipykernel_4216\2634338635.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'ethereum_price_lag_{lag}'] = df['ethereum_price'].shift(lag)
C:\Users\KYLEONEIL\AppData\Local\Temp\ipykernel_4216\2634338635.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

In [71]:
# train_size = int(len(data) * 0.8)
# train, test = data[:train_size], data[train_size:]

# # Features and target variable
# features = ['total_volume', 'market_cap'] + [f'price_lag_{lag}' for lag in range(1, 8)] + [f'price_month_lag_{lag}' for lag in range(1, 4)] + ['day_of_week', 'month', 'day_of_month']
# X_train, y_train = train[features], train['price']
# X_test, y_test = test[features], test['price']

In [72]:
features = ['total_volume', 'market_cap'] + \
           [f'price_lag_{lag}' for lag in range(1, 91)]  + \
           [f'price_month_lag_{lag}' for lag in range(1, 7)] + \
           ['day_of_week', 'month', 'day_of_month', 'year'] + \
           [f'ethereum_price_lag_{lag}' for lag in range(1, 5)]
        #    [f'ethereum_price_month_lag_{lag}' for lag in range(1, 2)] + \

# Remove the last row
df_without_last = df.iloc[:-1]

# Create X and y excluding the last row
X = df_without_last[features]
y = df_without_last['price']

# Extract the last row as test data
X_test = df.iloc[-1][features]
y_test = df.iloc[-1]['price']

last_row = df.iloc[[-1]]
X_test_last_row = last_row[features]

In [73]:
# Custom MAPE scorer
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

rmse_scores_linear_regression = []
mae_scores_linear_regression = []
r2_scores_linear_regression = []
mape_scores_linear_regression = []

rmse_scores_rf = []
mae_scores_rf = []
r2_scores_rf = []
mape_scores_rf = []

rmse_scores_dt = []
mae_scores_dt = []
r2_scores_dt = []
mape_scores_dt = []

rmse_scores_svm  = []
mae_scores_svm  = []
r2_scores_svm  = []
mape_scores_svm  = []
# Initialize TimeSeriesSplit
train_size = int(len(df_without_last) * 0.8)
xtrain, xtest = X[:train_size], X[train_size:]
ytrain, ytest = y[:train_size], y[train_size:]

rf = RandomForestRegressor()
    
cv_params = {
    'n_estimators' : [20, 50], 
    'max_depth': [4,5],
    'min_samples_leaf' : [1, 2, 5, 10], 
    'min_samples_split' : [2, 5, 100],
    'max_features' : [None],
    'max_samples' : [0.75, 0.9]
}
scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'r2': 'r2',
    'neg_mean_absolute_percentage_error': mape_scorer
}
grid_search = GridSearchCV(estimator=rf,
                        param_grid=cv_params,
                        cv=2,
                        verbose=1,
                        scoring=scoring,
                        refit='neg_mean_squared_error',
                        return_train_score=True)
# cv = GridSearchCV(rf, param_grid=cv_params, cv=5, refit='f1', n_jobs = -1, verbose = 1)
grid_search.fit(xtrain, ytrain)
rf_model = grid_search.best_estimator_
# rf_model.fit(xtrain, ytrain)
rf_predictions = rf_model.predict(xtest)
rf_rmse = np.sqrt(mean_squared_error(ytest, rf_predictions))
rf_mae = mean_absolute_error(ytest, rf_predictions)
rf_r2 = r2_score(ytest, rf_predictions)
rf_mape = mean_absolute_percentage_error(ytest, rf_predictions)

rmse_scores_rf.append(rf_rmse)
mae_scores_rf.append(rf_mae)
r2_scores_rf.append(rf_r2)
mape_scores_rf.append(rf_mape)

Fitting 2 folds for each of 96 candidates, totalling 192 fits


In [74]:
# Feature Scaling
scaler = StandardScaler()

# Define the SVR model
svm = SVR(kernel='rbf')

# Create a pipeline
pipeline = Pipeline([
    ('scaler', scaler),
    ('svm', svm)
])

# Define the hyperparameters to tune
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__epsilon': [0.01, 0.1, 0.2],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01]
}

# Time series split
tscv = TimeSeriesSplit(n_splits=5)

# Grid Search
grid_search_svm = GridSearchCV(pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error')
grid_search_svm.fit(X, y)
# Best parameters

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm', SVR())]),
             param_grid={'svm__C': [0.1, 1, 10, 100],
                         'svm__epsilon': [0.01, 0.1, 0.2],
                         'svm__gamma': ['scale', 'auto', 0.1, 0.01]},
             scoring='neg_mean_squared_error')

In [75]:

dt= DecisionTreeRegressor()
param_grid = {
    'max_depth': [4,5,10,15,20,25,30],
    'min_samples_leaf' : [1, 2, 5, 10, 20, 50, 100], 
    'min_samples_split' : [2, 5, 50, 100, 120],
    'max_features' : [None],
}
scoring = {
    'neg_mean_squared_error': 'neg_mean_squared_error',
    'neg_mean_absolute_error': 'neg_mean_absolute_error',
    'r2': 'r2',
    'neg_mean_absolute_percentage_error': mape_scorer
}
grid_search_dt = GridSearchCV(estimator=dt,
                       param_grid=param_grid,
                       cv=5,
                       scoring=scoring,
                       verbose=1,
                        refit='neg_mean_squared_error',
                       return_train_score=True)
grid_search_dt.fit(X, y)

Fitting 5 folds for each of 245 candidates, totalling 1225 fits


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 5, 10, 15, 20, 25, 30],
                         'max_features': [None],
                         'min_samples_leaf': [1, 2, 5, 10, 20, 50, 100],
                         'min_samples_split': [2, 5, 50, 100, 120]},
             refit='neg_mean_squared_error', return_train_score=True,
             scoring={'neg_mean_absolute_error': 'neg_mean_absolute_error',
                      'neg_mean_absolute_percentage_error': make_scorer(mean_absolute_percentage_error, greater_is_better=False, response_method='predict'),
                      'neg_mean_squared_error': 'neg_mean_squared_error',
                      'r2': 'r2'},
             verbose=1)

In [76]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit and evaluate your model here using the train and test sets
    # Example with Linear Regression:
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_predictions = lr_model.predict(X_test)
    lr_rmse = np.sqrt(mean_squared_error(y_test, lr_predictions))
    lr_mae = mean_absolute_error(y_test, lr_predictions)
    lr_r2 = r2_score(y_test, lr_predictions)
    lr_mape = mean_absolute_percentage_error(y_test, lr_predictions)
    
    rmse_scores_linear_regression.append(lr_rmse)
    mae_scores_linear_regression.append(lr_mae)
    r2_scores_linear_regression.append(lr_r2)
    mape_scores_linear_regression.append(lr_mape)


    # SVM
    best_model_svm = grid_search_svm.best_estimator_
    best_model_svm.fit(X_train, y_train)
    
    svm_predictions = best_model_svm.predict(X_test)
    svm_rmse = np.sqrt(mean_squared_error(y_test, svm_predictions))
    svm_mae = mean_absolute_error(y_test, svm_predictions)
    svm_r2 = r2_score(y_test, svm_predictions)
    svm_mape = mean_absolute_percentage_error(y_test, svm_predictions)
    
    rmse_scores_svm.append(svm_rmse)
    mae_scores_svm.append(svm_mae)
    r2_scores_svm.append(svm_r2)
    mape_scores_svm.append(svm_mape)


    # Decision Tree Model
    best_model_dt = grid_search_dt.best_estimator_
    best_model_dt.fit(X_train, y_train)
    dt_predictions = best_model_dt.predict(X_test)
    dt_rmse = np.sqrt(mean_squared_error(y_test, dt_predictions))
    dt_mae = mean_absolute_error(y_test, dt_predictions)
    dt_r2 = r2_score(y_test, dt_predictions)
    dt_mape = mean_absolute_percentage_error(y_test, dt_predictions)
    
    rmse_scores_dt.append(dt_rmse)
    mae_scores_dt.append(dt_mae)
    r2_scores_dt.append(dt_r2)
    mape_scores_dt.append(dt_mape)

    



In [77]:

lg_std_rmse = np.std(rmse_scores_linear_regression)
df_std_rmse = np.std(rmse_scores_dt)
dt_std_rmse = np.std(rmse_scores_rf)
svm_std_rmse = np.std(rmse_scores_svm)

print(f"Linear Regression RMSE: {np.mean(rmse_scores_linear_regression)}")
print(f'Linear RegressionStandard Deviation of RMSE: {lg_std_rmse}')
print(f"Linear Regression MAE: {np.mean(mae_scores_linear_regression)}")
print(f"Linear Regression R²: {np.mean(r2_scores_linear_regression)}")
print(f"Linear Regression MAPE: {np.mean(mape_scores_linear_regression)}")
print()
print(f"Random Forest RMSE: {np.mean(rmse_scores_rf)}")
print(f'Random Forest Standard Deviation of RMSE: {df_std_rmse}')
print(f"Random Forest MAE: {np.mean(mae_scores_rf)}")
print(f"Random Forest R²: {np.mean(r2_scores_rf)}")
print(f"Random Forest MAPE: {np.mean(mape_scores_rf)}")
print()
print(f"Decision Tree RMSE: {np.mean(rmse_scores_dt)}")
print(f'Decision Tree Standard Deviation of RMSE: {dt_std_rmse}')
print(f"Decision Tree MAE: {np.mean(mae_scores_dt)}")
print(f"Decision Tree R²: {np.mean(r2_scores_dt)}")
print(f"Decision Tree MAPE: {np.mean(mape_scores_dt)}")
print()
print(f"SVM RMSE: {np.mean(rmse_scores_svm)}")
print(f'SVM Standard Deviation of RMSE: {svm_std_rmse}')
print(f"SVM MAE: {np.mean(mae_scores_svm)}")
print(f"SVM R²: {np.mean(r2_scores_svm)}")
print(f"SVM MAPE: {np.mean(mape_scores_svm)}")

Linear Regression RMSE: 13.094918702906744
Linear RegressionStandard Deviation of RMSE: 10.162690266840317
Linear Regression MAE: 9.312246723483913
Linear Regression R²: 0.8034436631781368
Linear Regression MAPE: 13.628773761018405

Random Forest RMSE: 28.886383536204
Random Forest Standard Deviation of RMSE: 20.522685572390348
Random Forest MAE: 19.92828246240837
Random Forest R²: 0.6071263055724728
Random Forest MAPE: 22.379260094475374

Decision Tree RMSE: 29.470225559248934
Decision Tree Standard Deviation of RMSE: 0.0
Decision Tree MAE: 22.537702727042618
Decision Tree R²: -1.026825099109059
Decision Tree MAPE: 41.365700264859136

SVM RMSE: 33.189901486149246
SVM Standard Deviation of RMSE: 33.00661308043271
SVM MAE: 25.131719831963487
SVM R²: -0.23678401280912192
SVM MAPE: 34.65640485365019


In [78]:
print(df_without_last['price'].count())
print(df_without_last[features].count())

1356
total_volume            1356
market_cap              1356
price_lag_1             1356
price_lag_2             1356
price_lag_3             1356
                        ... 
year                    1356
ethereum_price_lag_1    1356
ethereum_price_lag_2    1356
ethereum_price_lag_3    1356
ethereum_price_lag_4    1356
Length: 106, dtype: int64


In [79]:
features = ['total_volume', 'market_cap'] + \
           [f'price_lag_{lag}' for lag in range(1, 31)]  + \
           [f'price_month_lag_{lag}' for lag in range(1, 7)] + \
           ['day_of_week', 'month', 'day_of_month', 'year'] + \
           [f'ethereum_price_lag_{lag}' for lag in range(1, 5)]
        #    [f'ethereum_price_month_lag_{lag}' for lag in range(1, 2)] + \

# Remove the last row
df_without_last = df.iloc[:-1]

# Create X and y excluding the last row
X = df_without_last[features]
y = df_without_last['price']


In [80]:
# Scaling the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Reshaping the data to fit into LSTM input shape (samples, timesteps, features)
X_scaled_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_reshaped, y_scaled, test_size=0.2, random_state=42)

# Building the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(X_train, y_train, batch_size=1, epochs=1)

# Predicting
predictions = model.predict(X_test)

# Inversing the scaling
predictions = scaler_y.inverse_transform(predictions)
y_test = scaler_y.inverse_transform(y_test)

# Evaluating the model
rmse = np.sqrt(np.mean(predictions - y_test)**2)
print(f"Root Mean Square Error: {rmse}")

c:\Users\KYLEONEIL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1084/1084 ━━━━━━━━━━━━━━━━━━━━ 2s 699us/step - loss: 0.0096
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Root Mean Square Error: 0.5105524074657204


In [84]:

rmse_list = []
mae_list = []
r2_list = []
mape_list = []
for train_index, test_index in tscv.split(X_scaled_reshaped):
    X_train, X_test = X_scaled_reshaped[train_index], X_scaled_reshaped[test_index]
    y_train, y_test = y_scaled[train_index], y_scaled[test_index]

    # Building the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compiling the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Training the model
    model.fit(X_train, y_train, batch_size=1, epochs=10)

    # Predicting
    predictions = model.predict(X_test)

    # Inversing the scaling
    predictions = scaler_y.inverse_transform(predictions)
    y_test = scaler_y.inverse_transform(y_test)

    # Evaluating the model
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    # Appending the metrics to the lists
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    mape_list.append(mape)

# Calculating mean and standard deviation of the metrics
lstm_rmse_mean = np.mean(rmse_list)
lstm_rmse_std = np.std(rmse_list)
lstm_mae_mean = np.mean(mae_list)
lstm_r2_mean = np.mean(r2_list)
lstm_mape_mean = np.mean(mape_list)

Epoch 1/10


c:\Users\KYLEONEIL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


226/226 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - loss: 5.8862e-05
Epoch 2/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 1.7021e-05
Epoch 3/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: 1.6286e-05
Epoch 4/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step - loss: 1.1567e-05
Epoch 5/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 1.4262e-05
Epoch 6/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 7.5229e-06
Epoch 7/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 6.8096e-06
Epoch 8/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 9.3712e-06
Epoch 9/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 7.1546e-06
Epoch 10/10
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 1.0552e-05
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch 1/10


c:\Users\KYLEONEIL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


452/452 ━━━━━━━━━━━━━━━━━━━━ 2s 734us/step - loss: 0.0093
Epoch 2/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 683us/step - loss: 0.0016
Epoch 3/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.0010
Epoch 4/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 0.0011  
Epoch 5/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - loss: 6.3904e-04
Epoch 6/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - loss: 0.0011
Epoch 7/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - loss: 4.2763e-04
Epoch 8/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - loss: 4.0622e-04
Epoch 9/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 7.2986e-04
Epoch 10/10
452/452 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - loss: 5.1515e-04
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Epoch 1/10


...

678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 703us/step - loss: 0.0237
Epoch 2/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - loss: 0.0019
Epoch 3/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step - loss: 0.0014
Epoch 4/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - loss: 0.0010
Epoch 5/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 0.0013
Epoch 6/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 807us/step - loss: 0.0011
Epoch 7/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 784us/step - loss: 8.5571e-04
Epoch 8/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 730us/step - loss: 7.7861e-04
Epoch 9/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 742us/step - loss: 0.0011
Epoch 10/10
678/678 ━━━━━━━━━━━━━━━━━━━━ 1s 737us/step - loss: 9.9297e-04
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Epoch 1/10


c:\Users\KYLEONEIL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


904/904 ━━━━━━━━━━━━━━━━━━━━ 2s 773us/step - loss: 0.0107
Epoch 2/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - loss: 0.0017
Epoch 3/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 782us/step - loss: 0.0014
Epoch 4/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 815us/step - loss: 0.0010
Epoch 5/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - loss: 0.0011
Epoch 6/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 766us/step - loss: 7.6332e-04
Epoch 7/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - loss: 8.1666e-04
Epoch 8/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - loss: 6.9240e-04
Epoch 9/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - loss: 8.1448e-04
Epoch 10/10
904/904 ━━━━━━━━━━━━━━━━━━━━ 1s 736us/step - loss: 0.0011
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch 1/10


c:\Users\KYLEONEIL\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1130/1130 ━━━━━━━━━━━━━━━━━━━━ 2s 690us/step - loss: 0.0105
Epoch 2/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - loss: 0.0018
Epoch 3/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 739us/step - loss: 0.0011
Epoch 4/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 710us/step - loss: 0.0011
Epoch 5/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 715us/step - loss: 0.0010
Epoch 6/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 739us/step - loss: 6.3074e-04
Epoch 7/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step - loss: 9.1218e-04
Epoch 8/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 726us/step - loss: 6.3355e-04
Epoch 9/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 688us/step - loss: 6.0011e-04
Epoch 10/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 1s 721us/step - loss: 6.6085e-04
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


In [85]:
lstm_rmse_mean = np.mean(rmse_list)
lstm_rmse_std = np.std(rmse_list)
lstm_mae_mean = np.mean(mae_list)
lstm_r2_mean = np.mean(r2_list)
lstm_mape_mean = np.mean(mape_list)

print(f"LSTM RMSE: {lstm_rmse_mean}")
print(f'LSTM Standard Deviation of RMSE: {lstm_rmse_std}')
print(f"LSTM MAE: {lstm_mae_mean}")
print(f"LSTM R²: {lstm_r2_mean}")
print(f"LSTM MAPE: {lstm_mape_mean}")

LSTM RMSE: 11.508734172026038
LSTM Standard Deviation of RMSE: 7.465800519833568
LSTM MAE: 8.073392684992939
LSTM R²: 0.5126631240147883
LSTM MAPE: 14.839295191598705


In [86]:

# Evaluation
lstm_rmse = np.sqrt(mean_squared_error(y_test, predictions))
lstm_rmse_std = np.std(lstm_rmse)
lstm_mae = mean_absolute_error(y_test, predictions)
lstm_r2 = r2_score(y_test, predictions)
lstm_mape = mean_absolute_percentage_error(y_test, predictions)

print(f"LSTM RMSE: {np.mean(lstm_rmse)}")
print(f'LSTM Standard Deviation of RMSE: {lstm_rmse_std}')
print(f"LSTM MAE: {np.mean(lstm_mae)}")
print(f"LSTM R²: {np.mean(lstm_r2)}")
print(f"LSTM MAPE: {np.mean(lstm_mape)}")
    

LSTM RMSE: 7.601445880138774
LSTM Standard Deviation of RMSE: 0.0
LSTM MAE: 6.472751045279943
LSTM R²: 0.9733393337575227
LSTM MAPE: 12.015628697925816
